In [1]:
import pandas as pd
import datetime

In [8]:
def gps_str_to_list(gps_data):
    gps_list = gps_data[1:-1].split(',')
    gps_list = [i.split() for i in gps_list]
    for i in range(len(gps_list)):
        # 去除方向数据
        daylist = pd.to_datetime(gps_list[i][4], unit='s', utc=True).tz_convert(
            "Asia/Shanghai").to_period("S").strftime('%Y/%m/%d %H:%M:%S').split()
        day = int(datetime.datetime.strptime(daylist[0],'%Y/%m/%d').strftime("%w"))
        time = (int(daylist[1].split(":")[0])*3600 + int(daylist[1].split(":")[1])*60 + int(daylist[1].split(":")[2]))/86400
        gps_list[i] = [float(gps_list[i][0]),
                       float(gps_list[i][1]),
                       round(float(gps_list[i][2])*3.6, 1),
                       float(gps_list[i][3]),
                       str(daylist[1].split(":")[0])+":"+str(daylist[1].split(":")[1]),
                       daylist[0],  # 日期
                       day,  # 周日-周六：0-6
                       time] # 归一化的当日时间
    return gps_list

In [11]:
def preprocess_tti(tti_data):
    tti_data['id_road'] = tti_data['id_road'].astype(str)
    tti_data['TTI'] = tti_data['TTI'].astype(float)
    tti_data['speed'] = tti_data['speed'].astype(float)
    tti_data['time'] = pd.to_datetime(
        tti_data['time'])#, infer_datetime_format=True)
    tti_data['day'] = tti_data['time'].apply(lambda x : str(x)[:-3])
    tti_data['weekday'] = (tti_data['time'].apply(lambda x : int(datetime.datetime.strptime(str(x).split()[0],'%Y-%m-%d').strftime("%w")))+1)/7
    tti_data['daytime'] = (tti_data['time'].apply(lambda x : int(str(x).split()[1].split(":")[0]))*3600 + tti_data['time'].apply(lambda x : int(str(x).split()[1].split(":")[1]))*60 + tti_data['time'].apply(lambda x : int(str(x).split()[1].split(":")[2])))/86400
    tti_data = tti_data.drop(columns = 'time')
    return tti_data

In [2]:
g1 = pd.read_csv('pred_gps_0-10000-1.csv') #g1就是用gps_str_to_list跑出来的前1w条测试集的数据
g1

,id_order,id_user,gps_records
0,1b16e3e220c596758adf339e8ea8374b,04c280acafb5e820a54cb27cb2cadb73,"[[114.04945, 22.61457, 59.1, 172.539993286, '1..."
1,02bf45a3b5c97ff9f8be1d4806cbca47,190243fe2cd69cc236517090c6e4ff08,"[[114.0132, 22.60036, 56.5, 47.3699989319, '17..."
2,9470eda1c5b02842e6e7b269ddefe72d,dfa935db5b01f355da1aa03eb2f1c7b7,"[[114.02856, 22.60801, -3.6, 0.0, '19:32', '20..."
3,21afcbd0f39de2c5d12f453fd77bcabb,4920846a373d97d74cf480060c0d0323,"[[114.01779, 22.58746, 52.5, 58.0078010559, '0..."
4,8eff795c0cf2761b876eacfc2c8dee50,e3d53ef49ed2ebe4f1aef6d5122c663e,"[[114.01857, 22.58749, 42.0, 43.2000007629, '0..."
5,d5efaa8f8e788b3dd27bd3b318253f94,d924f1647d8ff707b027572eb716d9d7,"[[114.04583, 22.60716, 1.6, 43.0999984741, '08..."
6,44670ef3f49a70700816fe5cc42567d6,6493ec9f8894b66edef3913f97e3718c,"[[114.01316, 22.60002, 40.9, 55.5200004578, '0..."
7,f90b25570d0f4a7ddb5262bdd4c6021d,f0e6edd71094e668b96c00d3e745b4dd,"[[114.02876, 22.60179, 48.0, 137.0, '08:25', '..."
8,d5666cc82d6e75fa15b5456067d51651,a834683a23061e91eda0290e58a34263,"[[114.05389, 22.58897, 73.2, 334.079986572, '0..."
9,b0aa630b1a6572355b7f502b24c32bd3,18e449f677ab150686c0e02f8b62f579,"[[114.03425, 22.60492, 10.9, 145.113998413, '0..."


In [3]:
g = g1
g

,id_order,id_user,gps_records
0,1b16e3e220c596758adf339e8ea8374b,04c280acafb5e820a54cb27cb2cadb73,"[[114.04945, 22.61457, 59.1, 172.539993286, '1..."
1,02bf45a3b5c97ff9f8be1d4806cbca47,190243fe2cd69cc236517090c6e4ff08,"[[114.0132, 22.60036, 56.5, 47.3699989319, '17..."
2,9470eda1c5b02842e6e7b269ddefe72d,dfa935db5b01f355da1aa03eb2f1c7b7,"[[114.02856, 22.60801, -3.6, 0.0, '19:32', '20..."
3,21afcbd0f39de2c5d12f453fd77bcabb,4920846a373d97d74cf480060c0d0323,"[[114.01779, 22.58746, 52.5, 58.0078010559, '0..."
4,8eff795c0cf2761b876eacfc2c8dee50,e3d53ef49ed2ebe4f1aef6d5122c663e,"[[114.01857, 22.58749, 42.0, 43.2000007629, '0..."
5,d5efaa8f8e788b3dd27bd3b318253f94,d924f1647d8ff707b027572eb716d9d7,"[[114.04583, 22.60716, 1.6, 43.0999984741, '08..."
6,44670ef3f49a70700816fe5cc42567d6,6493ec9f8894b66edef3913f97e3718c,"[[114.01316, 22.60002, 40.9, 55.5200004578, '0..."
7,f90b25570d0f4a7ddb5262bdd4c6021d,f0e6edd71094e668b96c00d3e745b4dd,"[[114.02876, 22.60179, 48.0, 137.0, '08:25', '..."
8,d5666cc82d6e75fa15b5456067d51651,a834683a23061e91eda0290e58a34263,"[[114.05389, 22.58897, 73.2, 334.079986572, '0..."
9,b0aa630b1a6572355b7f502b24c32bd3,18e449f677ab150686c0e02f8b62f579,"[[114.03425, 22.60492, 10.9, 145.113998413, '0..."


In [4]:
x = g['gps_records'].tolist()

In [5]:
x[0][2:-3].split("], [")[0].split(", ")

['114.04945',
 '22.61457',
 '59.1',
 '172.539993286',
 "'11:59'",
 "'2019/12/26'",
 '4',
 '0.4999189814814815']

In [6]:
gps1 = (float(g['gps_records'][5][2:-3].split("], [")[0].split(", ")[0]), float(g['gps_records'][5][2:-3].split("], [")[0].split(", ")[1]))
gps1

(114.04583, 22.60716)

In [9]:
tti_pred_path = "D:/third/ML/final/traffic1/toPredict_train_TTI.csv"

In [12]:
tti_pred_data = pd.read_csv(tti_pred_path)
tti_pred_data = preprocess_tti(tti_pred_data)
tti_pred_data

,id_road,TTI,speed,day,weekday,daytime
0,275911,1.05410,65.94030,2019-12-21 07:30,1.000000,0.312500
1,275911,1.06187,65.45780,2019-12-21 07:40,1.000000,0.319444
2,275911,1.04992,66.20310,2019-12-21 07:50,1.000000,0.326389
3,275911,1.04714,66.37870,2019-12-21 08:00,1.000000,0.333333
4,275911,1.08100,64.29970,2019-12-21 08:10,1.000000,0.340278
5,275911,1.09618,63.40920,2019-12-21 08:20,1.000000,0.347222
6,275911,1.13386,61.30200,2019-12-21 09:30,1.000000,0.395833
7,275911,1.19277,58.27450,2019-12-21 09:40,1.000000,0.402778
8,275911,1.23563,56.25280,2019-12-21 09:50,1.000000,0.409722
9,275911,1.29996,53.46930,2019-12-21 10:00,1.000000,0.416667


In [13]:
tti_pred_data['count'] = 0

In [7]:
def cars(x, y):
    for i in range(len(x)):
        j = 0
        temp1 = ''
        temp2 = ''
        while j < len(x[i][2:-3].split("], [")):
            gps = (float(x[i][2:-3].split("], [")[j].split(", ")[0]), float(x[i][2:-3].split("], [")[j].split(", ")[1]))
            for block in roads.keys():
                if in_block(gps, roads[block]):
                    road = get_road(float(x[i][2:-3].split("], [")[j].split(", ")[3]), block)
                    day = x[i][2:-3].split("], [")[j].split(", ")[5][1:5]+"-"+x[i][2:-3].split("], [")[j].split(", ")[5][6:8]+"-"+x[i][2:-3].split("], [")[j].split(", ")[5][9:-1]+" "+x[i][2:-3].split("], [")[j].split(", ")[4][1:-2]+"0"
                    if (road != temp1 or day != temp2) and temp1 != '' and temp2 != '':
                        idx = [k for k in y[y['id_road'].values == temp1].index if k in y[y['day'].values == day].index]
                        y.loc[idx[0], 'count'] += 1
                        print(idx[0])
                    temp1 = road
                    temp2 = day
            j += 3
    return y

In [17]:
tti_pred_data = cars(x, tti_pred_data)

4762
5266
2257
5286
5790
2766
2262
5286
5790
5286
5790
5286
1221
3237
1222
3238
5758
2231
3743
3239
4753
3247
3247
2238
2742
2238
2742
2239
2239
233
2257
749
4789
4789
4285
4789
4285
4789
4789
256
760
256
2272
5296
4287
4791
4288
4792
4802
4802
4298
4803
5307
5812
5308
777
1286
3302
4817
5321
5325
5829
2805
2301
4318
1805
1301
716
212
716
212
2229
3245
5765
5261
4762
1234
3251
2241
5276
3764
1749
1749
237
4778
5285
5789
4781
4280
4783
2774
2270
5294
2269
4288
4792
5296
4797
3790
3790
1271
5807
2783
2279
5303
3791
3287
3791
4803
2302
5327
5327
790
286
2302
5388
4892
3378
5898
4386
4890
5394
3381
5902


KeyboardInterrupt: 

In [15]:
roads = {('276183', '276184'): [(114.018188,22.588653),(114.027565,22.591505),(114.027758,22.590376),(114.017544,22.586295)],
        ('275911', '275912'): [(114.01726,22.603209),(114.017045,22.58946),(114.014041,22.589064),(114.015629,22.602099)],
         ('276240', '276241'): [(114.016444,22.606378),(114.025585,22.612479), (114.027516,22.610934),(114.017431,22.604001)],
         ('276264', '276265'): [(114.027677,22.592268), (114.026433,22.60463),(114.027968,22.604453), (114.034105,22.593478)],
         ('276268', '276269'): [(114.035053,22.603313),(114.022179,22.61603),(114.023552,22.61706),(114.036341,22.603828)],
         ('276738', '276737'): [(114.030436,22.602313), (114.025071,22.608316),(114.025994,22.608969),(114.03123,22.602729)]}

rec_road = {('276183', '276184'): [22.591219, 22.586757, 114.017324, 114.028209],('275911', '275912'): [22.603728, 22.586757, 114.014567, 114.017324],
            ('276240', '276241'): [22.610348, 22.603514, 114.015564, 114.023718], ('276264', '276265'): [22.606308, 22.59149, 114.026608, 114.034633],
    ('276268', '276269'): [22.608082, 22.599193, 114.031416, 114.039827],('276738', '276737'): [22.610044, 22.602214, 114.023188, 114.031664]}

# 1：西往东：[(0, 180)]
# 2：东往西：[(180, 360)]
# 3：南往北：[(0, 90), (270, 360)]
# 4：北往南：[(90, 270)]
roads_direction = {'276183': 1, '276184': 2, '275911': 3, '275912': 4, '276240': 1, '276241': 2, '276264': 3, '276265': 4, '276268': 3,'276269': 4,
    '276738': 3, '276737': 4}

def get_road(direction, roads):
    directions = [roads_direction[roads[0]], roads_direction[roads[1]]]
    if directions == [1, 2]:
        if 0 <= direction < 180:
            return roads[0]
        else:
            return roads[1]
    else:
        if 90 <= direction < 270:
            return roads[1]
        else:
            return roads[0]

def get_block_rec(block):
    # 上下左右
    block_rec = [max([i[1] for i in block]), min([i[1] for i in block]),min([i[0] for i in block]),max([i[0] for i in block])]
    return block_rec

In [16]:
def in_block(sample, block):
    block_rec = get_block_rec(block)
    # 初筛，上下左右
    x, y = sample
    if y > block_rec[0] or \
            y < block_rec[1] or \
            x < block_rec[2] or \
            x > block_rec[3]:
        return False
    # 细筛，计算四边形
    A, B, C, D = block
    # 顺时针的格子顶点，若逆时针方向则false
    a = (B[0]-A[0])*(y-A[1])-(B[1]-A[1])*(x-A[0])
    b = (C[0]-B[0])*(y-B[1])-(C[1]-B[1])*(x-B[0])
    c = (D[0]-C[0])*(y-C[1])-(D[1]-C[1])*(x-C[0])
    d = (A[0]-D[0])*(y-D[1])-(A[1]-D[1])*(x-D[0])
    if (a > 0 and b > 0 and c > 0 and d > 0) or \
            (a < 0 and b < 0 and c < 0 and d < 0):
        return True
    return False

In [ ]:
# 我想的是等所有数据算完，tti_pred_data的count列就是车辆数